## This notebook contains code for computing and plotting frequency maps, weight maps, and ILC maps (currently assumes the sky components are only CMB and tSZ).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import healpy as hp
import tempfile
import shutil
import os
import sys
sys.path.append('../../shared')
sys.path.append('../../needlet_ILC_pipeline')
from generate_maps import generate_freq_maps
from input import Info
from utils import get_naming_str, build_NILC_maps, spectral_response, GaussianNeedlets
from pyilc_interface import setup_pyilc, load_wt_maps, setup_pyilc_hilc

In [ ]:
font = {'size'   : 20, 'family':'STIXGeneral'}
plt.rcParams.update({
     'text.usetex': True,
     'font.family': 'serif',
     'font.sans-serif': ['Computer Modern'],
     'font.size':20})
plt.rc_context({'axes.autolimit_mode': 'round_numbers'})

# Read yaml, set up Info object

In [ ]:
sim = 0
yaml_file = '../../needlet_ILC_pipeline/example_yaml_files/laptop.yaml'
pipeline = 'NILC'
save = True

# read in the input file and set up relevant info object
inp = Info(yaml_file)
inp.ell_sum_max = inp.ellmax

ells = np.arange(inp.ellmax+1)
res = stats.binned_statistic(ells[2:], ells[2:], statistic='mean', bins=inp.Nbins)
mean_ells = np.round((res[1][:-1]+res[1][1:])/2)

naming_str = get_naming_str(inp, pipeline)

env = os.environ.copy()
split = 1

# Plot Needlet Filters

In [ ]:
inp.GN_FWHM_arcmin = [300., 120., 60.] 
# inp.GN_FWHM_arcmin = [600., 300., 120., 60., 30., 15.] 
inp.Nscales = len(inp.GN_FWHM_arcmin)+1
inp.Nscales = 4
# inp.GN_FWHM_arcmin = [300., 120., 60.] 
ell, filters = GaussianNeedlets(inp, taper_width=0)
for n in range(inp.Nscales):
    plt.plot(ells, filters[n], color='k')
plt.grid()
plt.xlim(0,inp.ellmax)
plt.xlabel(r'$\ell$')
plt.ylabel(r'$h_\ell^{(n)}$')
if save:
    plt.savefig('needlet_filters.pdf', bbox_inches='tight')

# Generate Maps, HILC Maps, and NILC Maps

In [ ]:
map_tmpdir = tempfile.mkdtemp(dir='.')
comp_spectra, comp_maps, noise_maps = generate_freq_maps(inp, sim=sim, save=True, scaling=None, pars=None, include_noise=True, map_tmpdir=map_tmpdir)
cmb_cl, tsz_cl = comp_spectra
cmb_map, tsz_map = comp_maps

Nsplits = 2
N_preserved_comps = 2
all_wt_maps = np.zeros((Nsplits, N_preserved_comps, inp.Nscales, len(inp.freqs), 12*inp.nside**2))
for split in [1,2]:
    
    # use for harmonic ilc
    pyilc_tmpdir_hilc = setup_pyilc_hilc(sim, split, inp, env, map_tmpdir, suppress_printing=False, pars=None) #set suppress_printing=False to debug pyilc runs
    if split == 1:
        CMB_HILC_map_s0 = 1.e-6*hp.read_map(f'{pyilc_tmpdir_hilc}/sim{sim}_split1needletILCmap_component_CMB.fits')
        tSZ_HILC_map_s0 = hp.read_map(f'{pyilc_tmpdir_hilc}/sim{sim}_split1needletILCmap_component_tSZ.fits')
    elif split == 2:
        CMB_HILC_map_s1 = 1.e-6*hp.read_map(f'{pyilc_tmpdir_hilc}/sim{sim}_split2needletILCmap_component_CMB.fits')
        tSZ_HILC_map_s1 = hp.read_map(f'{pyilc_tmpdir_hilc}/sim{sim}_split2needletILCmap_component_tSZ.fits')
    
    # use for needlet ilc
    pyilc_tmpdir_nilc = setup_pyilc(sim, split, inp, env, map_tmpdir, suppress_printing=False, pars=None) #set suppress_printing=False to debug pyilc runs
    CMB_wt_maps, tSZ_wt_maps = load_wt_maps(inp, sim, split, pyilc_tmpdir_nilc, pars=None) #load weight maps
    all_wt_maps[split-1] = np.array([CMB_wt_maps, tSZ_wt_maps])
    if split == 1:
        CMB_NILC_map_s0 = 1.e-6*hp.read_map(f'{pyilc_tmpdir_nilc}/sim{sim}_split1needletILCmap_component_CMB.fits')
        tSZ_NILC_map_s0 = hp.read_map(f'{pyilc_tmpdir_nilc}/sim{sim}_split1needletILCmap_component_tSZ.fits')
    elif split == 2:
        CMB_NILC_map_s1 = 1.e-6*hp.read_map(f'{pyilc_tmpdir_nilc}/sim{sim}_split2needletILCmap_component_CMB.fits')
        tSZ_NILC_map_s1 = hp.read_map(f'{pyilc_tmpdir_nilc}/sim{sim}_split2needletILCmap_component_tSZ.fits')
    
    shutil.rmtree(pyilc_tmpdir_hilc)
    shutil.rmtree(pyilc_tmpdir_nilc)
shutil.rmtree(map_tmpdir)

# Display NILC Weight Maps

In [ ]:
plt.rcParams.update({'font.size':11})

## Weight Maps for CMB-preserved NILC Map

In [ ]:
fig, axs = plt.subplots(inp.Nscales, len(inp.freqs), figsize=(8,6))
axs = axs.flatten()

ax = 0
for n in range(inp.Nscales):
    for i in range(len(inp.freqs)):
        plt.axes(axs[ax])
        hp.mollview(all_wt_maps[0][0][n][i], hold=True, title='', format='%.03g')
        if i == 0:
            plt.text(0.85, 0.5, f'Scale {n}\n{int(inp.freqs[i])} GHz', transform=axs[ax].transAxes,size=11)
        elif i == 1:
            plt.text(-0.04, 0.5, f'Scale {n}\n{int(inp.freqs[i])} GHz', transform=axs[ax].transAxes,size=11)
        ax += 1
if save:
    plt.savefig('cmb_wt_maps.pdf', bbox_inches='tight')

## Weight Maps for tSZ-preserved NILC Map

In [ ]:
fig, axs = plt.subplots(inp.Nscales, len(inp.freqs), figsize=(8,6))
axs = axs.flatten()

ax = 0
for n in range(inp.Nscales):
    for i in range(len(inp.freqs)):
        plt.axes(axs[ax])
        hp.mollview(all_wt_maps[0][1][n][i], hold=True, title='', format='%.03g', unit='[1/K]')
        if i == 0:
            plt.text(0.85, 0.5, f'Scale {n}\n{int(inp.freqs[i])} GHz', transform=axs[ax].transAxes,size=11)
        elif i == 1:
            plt.text(-0.04, 0.5, f'Scale {n}\n{int(inp.freqs[i])} GHz', transform=axs[ax].transAxes,size=11)
        ax += 1        
if save:
    plt.savefig('tsz_wt_maps.pdf', bbox_inches='tight')

# Compare input maps to HILC and NILC maps

In [ ]:
g90, g150 = spectral_response([90, 150], 'tsz')
map_0_s0 = cmb_map + g90*tsz_map + noise_maps[0,0]
map_1_s0 = cmb_map + g150*tsz_map + noise_maps[1,0]
map_0_s1 = cmb_map + g90*tsz_map + noise_maps[0,1]
map_1_s1 = cmb_map + g150*tsz_map + noise_maps[1,1]
CMB_NILC_map_s0_built, tSZ_NILC_map_s0_built = build_NILC_maps(inp, sim, filters, all_wt_maps[0], freq_maps=[map_0_s0, map_1_s0])
CMB_NILC_map_s1_built, tSZ_NILC_map_s1_built = build_NILC_maps(inp, sim, filters, all_wt_maps[1], freq_maps=[map_0_s1, map_1_s1])



In [ ]:
fig, axs = plt.subplots(2,3, figsize=(12,6))
axs = axs.flatten()
margins=None #(1,1,1,1)

plt.rcParams.update({
     'text.usetex': True,
     'font.family': 'serif',
     'font.sans-serif': ['Computer Modern'],
     'font.size':14})

for n, ax in enumerate(axs):
    plt.axes(ax)
    cmb_max = 0.0005
    if n==0:
        hp.mollview(cmb_map, fig=1, hold=True, title='Input CMB', format='%.03g', min=-cmb_max, max=cmb_max)
    elif n==1:
        hp.mollview(CMB_NILC_map_s0, fig=2, hold=True, title='CMB NILC', format='%.03g', min=-cmb_max, max=cmb_max)
    elif n==2:
        hp.mollview(CMB_HILC_map_s0, fig=2, hold=True, title='CMB HILC', format='%.03g', min=-cmb_max, max=cmb_max)
    elif n==3:
        hp.mollview(tsz_map, fig=2, hold=True, title=r'Input Amplified Compton-$y$', format='%.03g', min=0, max=0.0005)
    elif n==4:
        hp.mollview(tSZ_NILC_map_s0, fig=2, hold=True, title=r'Amplified Compton-$y$ NILC', format='%.03g', min=0, max=0.0005)
    else:
        hp.mollview(tSZ_HILC_map_s0, fig=2, hold=True, title=r'Amplified Compton-$y$ HILC', format='%.03g', min=0, max=0.0005)
    if n <= 2:
        plt.text(0.47, -0.02, r'$[\mathrm{K}]$', transform=ax.transAxes,size=14)

if save:
    plt.savefig('recon_maps.pdf', bbox_inches='tight')

plt.rcParams.update({
     'text.usetex': True,
     'font.family': 'serif',
     'font.sans-serif': ['Computer Modern'],
     'font.size':20})



# Compare Power Spectra

In [ ]:
to_dl = ells*(ells+1)/2/np.pi
plt.plot(ells[2:], (to_dl*hp.anafast(cmb_map, lmax=inp.ellmax))[2:], label='Input CMB')
plt.plot(ells[2:], (to_dl*hp.anafast(CMB_NILC_map_s0, CMB_NILC_map_s1, lmax=inp.ellmax))[2:], label='CMB NILC')
plt.plot(ells[2:], (to_dl*hp.anafast(CMB_HILC_map_s0, CMB_HILC_map_s1, lmax=inp.ellmax))[2:], label='CMB HILC')
plt.grid()
plt.legend(fontsize=12)
plt.xlim(0, inp.ellmax)
plt.xlabel(r'$\ell$')
plt.ylabel(r'$\ell(\ell+1)C_\ell/(2\pi)$ [$\mathrm{K}^2$]')

In [ ]:
to_dl = ells*(ells+1)/2/np.pi
plt.plot(ells, to_dl*hp.anafast(tsz_map, lmax=inp.ellmax), label='Input Amplified Compton-y')
plt.plot(ells, to_dl*hp.anafast(tSZ_NILC_map_s0, tSZ_NILC_map_s1, lmax=inp.ellmax), label='Amplified Compton-y NILC')
plt.plot(ells, to_dl*hp.anafast(tSZ_HILC_map_s0, tSZ_HILC_map_s1, lmax=inp.ellmax), label='Amplified Compton-y HILC')
plt.grid()
plt.legend(fontsize=12)
plt.xlim(0, inp.ellmax)
plt.xlabel(r'$\ell$')
plt.ylabel(r'$\ell(\ell+1)C_\ell/(2\pi)$')

In [ ]:
sys.path.append('../../harmonic_ILC_pipeline')
inp.delta_l = 10
inp.compute_weights_once = False
inp.Nbins = 249
inp.omit_central_ell = True
from hilc_analytic import get_freq_power_spec, get_data_vecs
Clij_new = get_freq_power_spec(inp, 0)
Clpq_new = get_data_vecs(inp, Clij_new)

In [ ]:
to_dl = ells*(ells+1)/2/np.pi
plt.plot(ells[2:], (to_dl*hp.anafast(CMB_HILC_map_s0, CMB_HILC_map_s1, lmax=inp.ellmax))[2:], label='CMB HILC (from map)', color='blue')
plt.plot(ells[2:], (to_dl[2:]*Clpq_new[0,0,0]), label='CMB HILC (analytic)', linestyle='dotted', color='lightblue')
plt.plot(ells[2:], (to_dl*hp.anafast(tSZ_HILC_map_s0, tSZ_HILC_map_s1, lmax=inp.ellmax))[2:], label=r'Amplified Compton-$y$ HILC (from map)', color='green')
plt.plot(ells[2:], (to_dl[2:]*Clpq_new[1,1,0]), label=r'Amplified Compton-$y$ HILC (analytic)', linestyle='dotted', color='lightgreen')
plt.grid()
plt.legend(fontsize=12)
plt.xlim(0, inp.ellmax)
plt.xlabel(r'$\ell$')
plt.ylabel(r'$\ell(\ell+1)C_\ell/(2\pi)$')
if save:
    plt.savefig('hilc_map_vs_analytic.pdf', bbox_inches='tight')

